Аналитики очень любят смотреть на графики. Однажды аналитик ВКонтакте пришёл на работу, открыл графики количества рекламных событий по дням (смотри приложение) и сразу же заметил сильный скачок данных за 11 апреля. 



Давайте поможем аналитику разобраться, что произошло. Какие-то ошибки в сборе статистик, дублирование данных? Или переживать не нужно и такой рост имеет разумное объяснение? 

Задача:
1. Воспроизведите график числа рекламных событий по дням. 
2. Найдите причину резкого увеличения количества рекламных событий и объясните, что произошло. 
З*. Предложите способ, который помог бы автоматически детектировать аномальные изменения метрик на графиках рекламной статистики. Иными словами, предложите алгоритм, который каждое утро анализирует данные за прошедшие сутки, и, если они сильно выбиваются из общего тренда, отправляет аналитику уведомление: на этом графике есть проблема за вчерашний день. Подумайте, как сделать детектор, который проверяет на аномалии каждый час, каждые пять минут в графике? 
Предложите вариант кода, который решает эту задачу. Реализовывать логику нотификации не нужно, только поиск аномального дня, часа, пятиминутки в данных рекламной статистики. 

Немного дополнительной информации о структуре рекламы ВКонтакте. Есть две основные группы рекламодателей: агентства и клиенты. Агентствами являются рекламные кабинеты юридических лиц. Среди клиентов же существуют как клиенты этих самых агентств, так и автономные рекламные кабинеты физических лиц, над которыми нет агентств. Рекламные кампании — это наборы рекламных объявлений. Они есть в каждом кабинете. 

In [44]:
import pandas as pd

In [46]:
df = pd.read_csv('D:\Documents\YandexDisk\__VK_jobs\Data.csv',parse_dates=True)

In [47]:
df.time = pd.to_datetime(df.time, unit='s')

In [48]:
df.head()

,time,event,ad_cost_type,has_video,agency_union_id,client_union_id,campaign_union_id,ad_id,platform,date
0,2019-04-08 17:22:25,report,CPM,0,no_agency,client_21074,campaign_27489,ad_27489,android,2019-04-08
1,2019-04-12 21:10:58,click,CPM,0,no_agency,client_22392,campaign_35893,ad_35893,android,2019-04-12
2,2019-04-16 08:16:41,click,CPM,0,no_agency,client_16836,campaign_26799,ad_26804,web,2019-04-16
3,2019-04-09 15:07:59,click,CPM,0,no_agency,client_22920,campaign_37146,ad_37146,android,2019-04-09
4,2019-04-11 07:11:57,click,CPM,0,no_agency,client_645,campaign_15893,ad_15944,web,2019-04-11


In [49]:
df2 = df.groupby('date')

In [50]:
df2.head()

,time,event,ad_cost_type,has_video,agency_union_id,client_union_id,campaign_union_id,ad_id,platform,date
0,2019-04-08 17:22:25,report,CPM,0,no_agency,client_21074,campaign_27489,ad_27489,android,2019-04-08
1,2019-04-12 21:10:58,click,CPM,0,no_agency,client_22392,campaign_35893,ad_35893,android,2019-04-12
2,2019-04-16 08:16:41,click,CPM,0,no_agency,client_16836,campaign_26799,ad_26804,web,2019-04-16
3,2019-04-09 15:07:59,click,CPM,0,no_agency,client_22920,campaign_37146,ad_37146,android,2019-04-09
4,2019-04-11 07:11:57,click,CPM,0,no_agency,client_645,campaign_15893,ad_15944,web,2019-04-11
5,2019-04-16 12:48:33,view,CPC,0,no_agency,client_265,campaign_13516,ad_13517,android,2019-04-16
6,2019-04-13 09:05:51,report,CPM,0,no_agency,client_22021,campaign_34637,ad_34651,mvk,2019-04-13
7,2019-04-05 22:24:03,view,CPM,0,agency_155,client_105617,campaign_106015,ad_106015,iphone,2019-04-05
8,2019-04-10 23:57:14,view,CPM,0,no_agency,client_264,campaign_13491,ad_13513,android,2019-04-10
9,2019-04-08 10:31:06,view,CPM,0,no_agency,client_1728,campaign_19254,ad_19255,android,2019-04-08


In [ ]:
df.plot(x='date',y=)